In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout,GRU
from keras.optimizers import Adam, RMSprop
from keras.losses import binary_crossentropy
from keras.metrics import accuracy

In [42]:
def normalize_data(df):
    scaler = MinMaxScaler()
    for i, column in enumerate(data_df.columns):
        df[data_df.columns[i]] = scaler.fit_transform(df[data_df.columns[i]].values.reshape(-1,1))
    return df

In [43]:
def generator(data, lookback=10, delay=5, min_index=0, max_index=None):
    data = data.values #DataFrame을 넘파이로 바꾸기
    max_index = len(data) - delay -1 #4439
    i = min_index + lookback #index10부터 
    samples = np.zeros((len(range(max_index-i+1)), lookback, data.shape[-1])) #(4430, 10, 33)
    targets = np.zeros((len(range(max_index-i+1)),)) #(4430,)
    while i <= max_index:
        indices = range(i-lookback, i)
        samples[i-lookback] = data[indices]
        if data[i+delay][0] - data[i][0] >= 0:
            targets[i-lookback] = 1
        else:
            targets[i-lookback] = 0
        i+=1
    return samples, targets

In [44]:
def split_data(sample, target, train_size_percentage=80, shuffle=True):
    if shuffle is True:
        index = np.arange(sample.shape[0])
        np.random.shuffle(index)
        sample = sample[index]
        target = target[index]
    train_size = train_size_percentage*sample.shape[0]//100
    test_size = sample.shape[0] - train_size
    
    x_train = sample[:train_size]
    y_train = target[:train_size]
    x_test = sample[train_size:]
    y_test = target[train_size:]
    
    return x_train, y_train, x_test, y_test  

In [45]:
def eval_acc(models, x_test_data, y_test_data, threshold=0.5):
    scores = models.predict(x_test_data)
    classified_scores=[]
    for score in scores:
        if score >= threshold:
            classified_scores.append(1)
        else:
            classified_scores.append(0)
    
    acc=0
    for i, classified_score in enumerate(classified_scores):
        if classified_score== y_test_data[i]:
            acc+=1
    accuracy_score = acc/len(y_test_data)*100
    
    return accuracy_score

In [46]:
from sklearn.metrics import f1_score

In [47]:
a_list = ['수출국(칠레)', '산업생산[스페인]', '본원소득수입[브라질]', 'OECD 경기선행지수[브라질]', '무역수지[그리스]', '도매재고율[미국]', '무역수지[오스트리아]', '본원소득지급[브라질]', '무역수지[포르투갈]', '재고지수[미국]', '실업률[오스트리아_NSA]', '건설업신뢰지수(EURO17개국)', '배송지수[미국]', '무역수지[룩셈부르크]', '산업생산[브라질, NSA]', '산업생산[룩셈부르크]', '무역수지[아일랜드]', '경기체감지수(EURO17개국)', '수출국(멕시코)', '산업생산[그리스]', 'NAHB주택시장지수[미국]', '자동차판매[AutoSales][미국]', '무역수지[말레이시아]']
a_final = []

In [48]:
a=1

data_df = pd.read_csv('final2.csv')
data_df = data_df.drop(['날짜','변동률','연중최고가종목수','연중최저가종목수'], axis=1, inplace=False)
normalized_data_df = normalize_data(data_df)
samples, targets = generator(normalized_data_df, min_index=0, max_index=3500)
x_train, y_train, x_test, y_test = split_data(samples, targets)
model = Sequential()
model.add(GRU(32,input_shape=(x_train.shape[1], x_train.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(64, return_sequences=True))
model.add(Dropout(0.5))
model.add(GRU(64))
model.add(Dense(32, activation='relu'))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam')

model.fit(x_train,y_train,batch_size=64,epochs=25,verbose=1,validation_split=0.25)
number1 = eval_acc(model, x_test, y_test)
    
for target in a_list:
    print("number: ",a)
    data_df = pd.read_csv('final2.csv')
    data_df = data_df.drop(['날짜','변동률','연중최고가종목수','연중최저가종목수', target], axis=1, inplace=False)
    normalized_data_df = normalize_data(data_df)
    samples, targets = generator(normalized_data_df, min_index=0, max_index=3500)
    x_train, y_train, x_test, y_test = split_data(samples, targets)
    model = Sequential()
    model.add(GRU(32,input_shape=(x_train.shape[1], x_train.shape[2]), return_sequences=True))
    model.add(Dropout(0.2))
    model.add(GRU(64, return_sequences=True))
    model.add(Dropout(0.5))
    model.add(GRU(64))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1,activation='sigmoid'))

    model.compile(loss='binary_crossentropy',optimizer='adam')

    model.fit(x_train,y_train,batch_size=64,epochs=25,verbose=1,validation_split=0.25)
    number2 = eval_acc(model, x_test, y_test)
    
    if number2 > number1:
        a_final.append(target)
    a+=1
    print("number1 :",number1)
    print("number2 :",number2)
    print("now target: ", a_final)
print(a_final)

Train on 2658 samples, validate on 886 samples
Epoch 1/25
2658/2658 [==============================] - 9s 4ms/step - loss: 0.6896 - val_loss: 0.6933
Epoch 2/25
2658/2658 [==============================] - 8s 3ms/step - loss: 0.6859 - val_loss: 0.6852
Epoch 3/25
2658/2658 [==============================] - 7s 3ms/step - loss: 0.6829 - val_loss: 0.6914
Epoch 4/25
2658/2658 [==============================] - 7s 3ms/step - loss: 0.6802 - val_loss: 0.6874
Epoch 5/25
2658/2658 [==============================] - 8s 3ms/step - loss: 0.6887 - val_loss: 0.6841
Epoch 6/25
2658/2658 [==============================] - 8s 3ms/step - loss: 0.6791 - val_loss: 0.6809
Epoch 7/25
2658/2658 [==============================] - 7s 3ms/step - loss: 0.6765 - val_loss: 0.6801
Epoch 8/25
2658/2658 [==============================] - 5s 2ms/step - loss: 0.6744 - val_loss: 0.6724
Epoch 9/25
2658/2658 [==============================] - 4s 2ms/step - loss: 0.6720 - val_loss: 0.6695
Epoch 10/25
2658/2658 [============

Epoch 3/25
2658/2658 [==============================] - 4s 2ms/step - loss: 0.6822 - val_loss: 0.6771
Epoch 4/25
2658/2658 [==============================] - 5s 2ms/step - loss: 0.6801 - val_loss: 0.6709
Epoch 5/25
2658/2658 [==============================] - 4s 2ms/step - loss: 0.6724 - val_loss: 0.6736
Epoch 6/25
2658/2658 [==============================] - 5s 2ms/step - loss: 0.6740 - val_loss: 0.6724
Epoch 7/25
2658/2658 [==============================] - 5s 2ms/step - loss: 0.6729 - val_loss: 0.6723
Epoch 8/25
2658/2658 [==============================] - 4s 2ms/step - loss: 0.6714 - val_loss: 0.6676
Epoch 9/25
2658/2658 [==============================] - 5s 2ms/step - loss: 0.6709 - val_loss: 0.6685
Epoch 10/25
2658/2658 [==============================] - 4s 2ms/step - loss: 0.6656 - val_loss: 0.6699
Epoch 11/25
2658/2658 [==============================] - 5s 2ms/step - loss: 0.6715 - val_loss: 0.6756
Epoch 12/25
2658/2658 [==============================] - 6s 2ms/step - loss: 0.6

2658/2658 [==============================] - 5s 2ms/step - loss: 0.6854 - val_loss: 0.6856
Epoch 3/25
2658/2658 [==============================] - 5s 2ms/step - loss: 0.6840 - val_loss: 0.6830
Epoch 4/25
2658/2658 [==============================] - 5s 2ms/step - loss: 0.6782 - val_loss: 0.6832
Epoch 5/25
2658/2658 [==============================] - 5s 2ms/step - loss: 0.6752 - val_loss: 0.7019
Epoch 6/25
2658/2658 [==============================] - 5s 2ms/step - loss: 0.6780 - val_loss: 0.6785
Epoch 7/25
2658/2658 [==============================] - 5s 2ms/step - loss: 0.6700 - val_loss: 0.6787
Epoch 8/25
2658/2658 [==============================] - 5s 2ms/step - loss: 0.6690 - val_loss: 0.6766
Epoch 9/25
2658/2658 [==============================] - 4s 2ms/step - loss: 0.6702 - val_loss: 0.6786
Epoch 10/25
2658/2658 [==============================] - 5s 2ms/step - loss: 0.6656 - val_loss: 0.6755
Epoch 11/25
2658/2658 [==============================] - 5s 2ms/step - loss: 0.6666 - val_lo

Train on 2658 samples, validate on 886 samples
Epoch 1/25
2658/2658 [==============================] - 9s 3ms/step - loss: 0.6898 - val_loss: 0.6969
Epoch 2/25
2658/2658 [==============================] - 4s 2ms/step - loss: 0.6885 - val_loss: 0.6840
Epoch 3/25
2658/2658 [==============================] - 5s 2ms/step - loss: 0.6842 - val_loss: 0.6830
Epoch 4/25
2658/2658 [==============================] - 5s 2ms/step - loss: 0.6827 - val_loss: 0.6882
Epoch 5/25
2658/2658 [==============================] - 5s 2ms/step - loss: 0.6827 - val_loss: 0.6787
Epoch 6/25
2658/2658 [==============================] - 5s 2ms/step - loss: 0.6794 - val_loss: 0.6785
Epoch 7/25
2658/2658 [==============================] - 5s 2ms/step - loss: 0.6791 - val_loss: 0.6752
Epoch 8/25
2658/2658 [==============================] - 5s 2ms/step - loss: 0.6791 - val_loss: 0.6747
Epoch 9/25
2658/2658 [==============================] - 6s 2ms/step - loss: 0.6723 - val_loss: 0.6647
Epoch 10/25
2658/2658 [============

2658/2658 [==============================] - 5s 2ms/step - loss: 0.6537 - val_loss: 0.6650
Epoch 22/25
2658/2658 [==============================] - 5s 2ms/step - loss: 0.6534 - val_loss: 0.6694
Epoch 23/25
2658/2658 [==============================] - 4s 2ms/step - loss: 0.6519 - val_loss: 0.6647
Epoch 24/25
2658/2658 [==============================] - 5s 2ms/step - loss: 0.6500 - val_loss: 0.6601
Epoch 25/25
2658/2658 [==============================] - 5s 2ms/step - loss: 0.6455 - val_loss: 0.6575
number1 : 60.496613995485326
number2 : 58.465011286681715
now target:  ['산업생산[스페인]', '무역수지[오스트리아]', '본원소득지급[브라질]', '재고지수[미국]', '실업률[오스트리아_NSA]', '건설업신뢰지수(EURO17개국)', '무역수지[룩셈부르크]', '산업생산[브라질, NSA]', '산업생산[룩셈부르크]', '경기체감지수(EURO17개국)', '수출국(멕시코)']
number:  21
Train on 2658 samples, validate on 886 samples
Epoch 1/25
2658/2658 [==============================] - 9s 3ms/step - loss: 0.6903 - val_loss: 0.6850
Epoch 2/25
2658/2658 [==============================] - 5s 2ms/step - loss: 0.6855 - val_l

2658/2658 [==============================] - 5s 2ms/step - loss: 0.6627 - val_loss: 0.6687
Epoch 18/25
2658/2658 [==============================] - 6s 2ms/step - loss: 0.6619 - val_loss: 0.6673
Epoch 19/25
2658/2658 [==============================] - 5s 2ms/step - loss: 0.6618 - val_loss: 0.6779
Epoch 20/25
2658/2658 [==============================] - 5s 2ms/step - loss: 0.6627 - val_loss: 0.6752
Epoch 21/25
2658/2658 [==============================] - 5s 2ms/step - loss: 0.6600 - val_loss: 0.6673
Epoch 22/25
2658/2658 [==============================] - 5s 2ms/step - loss: 0.6573 - val_loss: 0.6686
Epoch 23/25
2658/2658 [==============================] - 5s 2ms/step - loss: 0.6584 - val_loss: 0.6641
Epoch 24/25
2658/2658 [==============================] - 5s 2ms/step - loss: 0.6543 - val_loss: 0.6695
Epoch 25/25
2658/2658 [==============================] - 5s 2ms/step - loss: 0.6633 - val_loss: 0.6671
number1 : 60.496613995485326
number2 : 60.270880361173816
now target:  ['산업생산[스페인]', 

In [49]:
print(a_final)

['산업생산[스페인]', '무역수지[오스트리아]', '본원소득지급[브라질]', '재고지수[미국]', '실업률[오스트리아_NSA]', '건설업신뢰지수(EURO17개국)', '무역수지[룩셈부르크]', '산업생산[브라질, NSA]', '산업생산[룩셈부르크]', '경기체감지수(EURO17개국)', '수출국(멕시코)']


In [50]:
print(a_list)

['수출국(칠레)', '산업생산[스페인]', '본원소득수입[브라질]', 'OECD 경기선행지수[브라질]', '무역수지[그리스]', '도매재고율[미국]', '무역수지[오스트리아]', '본원소득지급[브라질]', '무역수지[포르투갈]', '재고지수[미국]', '실업률[오스트리아_NSA]', '건설업신뢰지수(EURO17개국)', '배송지수[미국]', '무역수지[룩셈부르크]', '산업생산[브라질, NSA]', '산업생산[룩셈부르크]', '무역수지[아일랜드]', '경기체감지수(EURO17개국)', '수출국(멕시코)', '산업생산[그리스]', 'NAHB주택시장지수[미국]', '자동차판매[AutoSales][미국]', '무역수지[말레이시아]']


In [51]:
print(len(a_final))

11


In [52]:
data_df = pd.read_csv('final2.csv')
data_df = data_df.drop(['날짜','변동률','연중최고가종목수','연중최저가종목수'], axis=1, inplace=False)
data_df = data_df.drop(a_final, axis=1, inplace=False)
data_df.to_csv("final3.csv",index=False)

In [127]:
data_df = pd.read_csv('final2.csv')
data_df = data_df.drop(['날짜','변동률','연중최고가종목수','연중최저가종목수'], axis=1, inplace=False)
data_df = data_df.drop(a_list, axis=1, inplace=False)
data_df.head()

,지수종가,종가이동평균_5,거래량,거래량이동평균_5,자본금,전체종목수,상승종목수,하락종목수,보합종목수,상한종목수,...,월별 소비자물가등락률(전년비),이전소득수지,자본금융수지[브라질],자본수지,장단기금리차(%p)(선행),재고순환지표(%p)(선행),재정수지[미국],제조업평균가동률지수(계절조정),제조업평균가동률지수(원지수),직접투자[브라질]
0,586.65,553.580,558645100,19.873264,84853590,902,533,278,45,72,...,3.4,66.875,5.404533,-10.9655,1.35,-10.8,76379.0,95.1,86.9,-1568.746882
1,589.92,567.374,558967700,20.016030,84881174,902,527,282,49,64,...,3.4,66.875,5.404533,-10.9655,1.35,-10.8,76379.0,95.1,86.9,-1568.746882
2,560.81,575.250,642378200,20.137292,84891369,902,137,698,23,36,...,3.4,66.875,5.404533,-10.9655,1.35,-10.8,76379.0,95.1,86.9,-1568.746882
3,561.79,576.004,518788600,20.163623,84906240,902,574,242,43,87,...,3.4,66.875,5.404533,-10.9655,1.35,-10.8,76379.0,95.1,86.9,-1568.746882
4,587.87,577.408,713787700,20.209960,84909240,902,494,298,68,27,...,3.4,66.875,5.404533,-10.9655,1.35,-10.8,76379.0,95.1,86.9,-1568.746882


In [128]:
normalized_data_df = normalize_data(data_df)
samples, targets = generator(normalized_data_df, min_index=0, max_index=3500)
x_train, y_train, x_test, y_test = split_data(samples, targets)
model = Sequential()
model.add(GRU(32,input_shape=(x_train.shape[1], x_train.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(64, return_sequences=True))
model.add(Dropout(0.5))
model.add(GRU(64))
model.add(Dense(32, activation='relu'))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam')

model.fit(x_train,y_train,batch_size=64,epochs=25,verbose=1,validation_split=0.25)
number1 = eval_acc(model, x_test, y_test)
    

Train on 2658 samples, validate on 886 samples
Epoch 1/25
2658/2658 [==============================] - 9s 3ms/step - loss: 0.6867 - val_loss: 0.6984
Epoch 2/25
2658/2658 [==============================] - 4s 2ms/step - loss: 0.6802 - val_loss: 0.6948
Epoch 3/25
2658/2658 [==============================] - 4s 2ms/step - loss: 0.6760 - val_loss: 0.6877
Epoch 4/25
2658/2658 [==============================] - 4s 2ms/step - loss: 0.6725 - val_loss: 0.6694
Epoch 5/25
2658/2658 [==============================] - 4s 2ms/step - loss: 0.6710 - val_loss: 0.6766
Epoch 6/25
2658/2658 [==============================] - 4s 2ms/step - loss: 0.6680 - val_loss: 0.6673
Epoch 7/25
2658/2658 [==============================] - 4s 2ms/step - loss: 0.6654 - val_loss: 0.6651
Epoch 8/25
2658/2658 [==============================] - 5s 2ms/step - loss: 0.6665 - val_loss: 0.6706
Epoch 9/25
2658/2658 [==============================] - 5s 2ms/step - loss: 0.6689 - val_loss: 0.6837
Epoch 10/25
2658/2658 [============

In [129]:
print(number1)

62.07674943566591


In [130]:
scores = model.predict(x_test)
classified_scores=[]
for score in scores:
    if score >= 0.5:
        classified_scores.append(1)
    else:
        classified_scores.append(0)
print(classified_scores)
print(len(classified_scores))

[1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 

In [131]:
df = pd.DataFrame()
data_df = pd.read_csv('final2.csv')
df['x'] = data_df['지수종가'][-886:]
df['y']=classified_scores

In [132]:
df.head(-1)

,x,y
3559,2146.10,1
3560,2143.50,1
3561,2107.50,1
3562,2110.89,1
3563,2114.80,0
...,...,...
4439,2060.12,1
4440,2061.49,1
4441,2055.01,1
4442,2028.01,1


In [133]:
money = 100000000
stock = 0
aa=[]
for j in range(5):
    money = 100000000
    stock = 0
    for i in range(j,886,5):
        a = []
        x=df['x'].iloc[i]
        y=df['y'].iloc[i]
        if y == 1 and money>0:
            stock = money/x
            money = 0
        if y == 0 and money==0:
            money = stock*x
            stock = 0
        a.append(money)
        a.append(stock)
    if a[0]==0:
        print(a[1]*x)
        aa.append(a[1]*x)
    else:
        print(a[0])
        aa.append(a[0])

85307040.19413753
109980059.82649647
96248631.92942788
103954099.05796026
109993413.88386486


In [134]:
print(sum(aa)/5000000)

101.0966489783774


In [135]:
print(df['x'].iloc[-1]*100/df['x'].iloc[0])

95.10460835934953
